## Contrarian Strategy

Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

Fetching the data source and converting to pandas dataframe

In [2]:
data = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')

Getting details about the asset data by analysing the dataframe

In [3]:
data.head(10)

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
2018-01-03 04:00:00+00:00,1.203825
2018-01-03 10:00:00+00:00,1.202355
2018-01-03 16:00:00+00:00,1.201445
2018-01-03 22:00:00+00:00,1.201450


In [4]:
data.tail(10)

,Close
time,
2019-12-26 16:00:00+00:00,1.109655
2019-12-26 22:00:00+00:00,1.111890
2019-12-27 04:00:00+00:00,1.113860
2019-12-27 10:00:00+00:00,1.116300
2019-12-27 16:00:00+00:00,1.117580
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095
2019-12-30 16:00:00+00:00,1.119920


In [5]:
data.Close.to_frame()

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
...,...
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095


Getting more statistical insight into the dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2070 entries, 2018-01-01 22:00:00+00:00 to 2019-12-30 22:00:00+00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2070 non-null   float64
dtypes: float64(1)
memory usage: 32.3 KB


In [7]:
data.describe()

,Close
count,2070.000000
mean,1.150400
std,0.041544
min,1.088635
25%,1.119714
50%,1.137698
75%,1.169170
max,1.253920


Since the only attribute we need is closing price
we modify the data frame

In [8]:
data = data.Close.to_frame()

In [9]:
data.head(10)

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
2018-01-03 04:00:00+00:00,1.203825
2018-01-03 10:00:00+00:00,1.202355
2018-01-03 16:00:00+00:00,1.201445
2018-01-03 22:00:00+00:00,1.201450


We plot the closing price of our asset

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

We proceed to define baseline and compute returns !needs improvement!

In [12]:
data['returns'] = np.log(data.div(data.shift(1)))

In [14]:
data.dropna(inplace=True)

In [15]:
data

,Close,returns
time,,
2018-01-02 04:00:00+00:00,1.207055,0.004858
2018-01-02 10:00:00+00:00,1.204440,-0.002169
2018-01-02 16:00:00+00:00,1.205800,0.001129
2018-01-02 22:00:00+00:00,1.204690,-0.000921
2018-01-03 04:00:00+00:00,1.203825,-0.000718
...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092
2019-12-30 04:00:00+00:00,1.119940,0.000018
2019-12-30 10:00:00+00:00,1.120095,0.000138


Defining a simple contrarian strategy (window = 3)

In [16]:
window = 3

In [17]:
data['returns'].rolling(window).mean()

time
2018-01-02 04:00:00+00:00             NaN
2018-01-02 10:00:00+00:00             NaN
2018-01-02 16:00:00+00:00    1.272676e-03
2018-01-02 22:00:00+00:00   -6.537454e-04
2018-01-03 04:00:00+00:00   -1.702470e-04
                                 ...     
2019-12-29 22:00:00+00:00    1.808598e-03
2019-12-30 04:00:00+00:00    1.085156e-03
2019-12-30 10:00:00+00:00    7.492899e-04
2019-12-30 16:00:00+00:00   -1.484453e-17
2019-12-30 22:00:00+00:00    1.234956e-04
Name: returns, Length: 2069, dtype: float64

In [18]:
data['position'] = -np.sign(data['returns'].rolling(window).mean()) # contrarian (minus sign)

In [19]:
data

,Close,returns,position
time,,,
2018-01-02 04:00:00+00:00,1.207055,0.004858,NaN
2018-01-02 10:00:00+00:00,1.204440,-0.002169,NaN
2018-01-02 16:00:00+00:00,1.205800,0.001129,-1.0
2018-01-02 22:00:00+00:00,1.204690,-0.000921,1.0
2018-01-03 04:00:00+00:00,1.203825,-0.000718,1.0
...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,-1.0
2019-12-30 04:00:00+00:00,1.119940,0.000018,-1.0
2019-12-30 10:00:00+00:00,1.120095,0.000138,-1.0


In [20]:
data['strategy'] = data.position.shift(1) * data['returns']

In [22]:
data.dropna(inplace=True)

In [23]:
data

,Close,returns,position,strategy
time,,,,
2018-01-02 22:00:00+00:00,1.204690,-0.000921,1.0,0.000921
2018-01-03 04:00:00+00:00,1.203825,-0.000718,1.0,-0.000718
2018-01-03 10:00:00+00:00,1.202355,-0.001222,1.0,-0.001222
2018-01-03 16:00:00+00:00,1.201445,-0.000757,1.0,-0.000757
2018-01-03 22:00:00+00:00,1.201450,0.000004,1.0,0.000004
...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,-1.0,-0.002092
2019-12-30 04:00:00+00:00,1.119940,0.000018,-1.0,-0.000018
2019-12-30 10:00:00+00:00,1.120095,0.000138,-1.0,-0.000138


In [24]:
data[['returns', 'strategy']].sum()

returns    -0.073498
strategy    0.096236
dtype: float64

In [25]:
data['creturns'] = data['returns'].cumsum().apply(np.exp)
data['cstrategy'] = data['strategy'].cumsum().apply(np.exp)

In [26]:
data

,Close,returns,position,strategy,creturns,cstrategy
time,,,,,,
2018-01-02 22:00:00+00:00,1.204690,-0.000921,1.0,0.000921,0.999079,1.000921
2018-01-03 04:00:00+00:00,1.203825,-0.000718,1.0,-0.000718,0.998362,1.000203
2018-01-03 10:00:00+00:00,1.202355,-0.001222,1.0,-0.001222,0.997143,0.998981
2018-01-03 16:00:00+00:00,1.201445,-0.000757,1.0,-0.000757,0.996388,0.998225
2018-01-03 22:00:00+00:00,1.201450,0.000004,1.0,0.000004,0.996392,0.998229
...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,-1.0,-0.002092,0.928778,1.100591
2019-12-30 04:00:00+00:00,1.119940,0.000018,-1.0,-0.000018,0.928794,1.100572
2019-12-30 10:00:00+00:00,1.120095,0.000138,-1.0,-0.000138,0.928923,1.100420


In [27]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

Taking trading costs into consideration

In [28]:
ptc = 0.00007

In [29]:
data['trades'] = data.position.diff().fillna(0).abs()

In [30]:
data

,Close,returns,position,strategy,creturns,cstrategy,trades
time,,,,,,,
2018-01-02 22:00:00+00:00,1.204690,-0.000921,1.0,0.000921,0.999079,1.000921,0.0
2018-01-03 04:00:00+00:00,1.203825,-0.000718,1.0,-0.000718,0.998362,1.000203,0.0
2018-01-03 10:00:00+00:00,1.202355,-0.001222,1.0,-0.001222,0.997143,0.998981,0.0
2018-01-03 16:00:00+00:00,1.201445,-0.000757,1.0,-0.000757,0.996388,0.998225,0.0
2018-01-03 22:00:00+00:00,1.201450,0.000004,1.0,0.000004,0.996392,0.998229,0.0
...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,-1.0,-0.002092,0.928778,1.100591,0.0
2019-12-30 04:00:00+00:00,1.119940,0.000018,-1.0,-0.000018,0.928794,1.100572,0.0
2019-12-30 10:00:00+00:00,1.120095,0.000138,-1.0,-0.000138,0.928923,1.100420,0.0


In [35]:
data.trades.value_counts()

0.0    1513
2.0     553
Name: trades, dtype: int64

In [36]:
data['strategy_net'] = data.strategy - data.trades * ptc

In [37]:
data['cstrategy_net'] = data.strategy_net.cumsum().apply(np.exp)

In [38]:
data

,Close,returns,position,strategy,creturns,cstrategy,trades,strategy_net,cstrategy_net
time,,,,,,,,,
2018-01-02 22:00:00+00:00,1.204690,-0.000921,1.0,0.000921,0.999079,1.000921,0.0,0.000921,1.000921
2018-01-03 04:00:00+00:00,1.203825,-0.000718,1.0,-0.000718,0.998362,1.000203,0.0,-0.000718,1.000203
2018-01-03 10:00:00+00:00,1.202355,-0.001222,1.0,-0.001222,0.997143,0.998981,0.0,-0.001222,0.998981
2018-01-03 16:00:00+00:00,1.201445,-0.000757,1.0,-0.000757,0.996388,0.998225,0.0,-0.000757,0.998225
2018-01-03 22:00:00+00:00,1.201450,0.000004,1.0,0.000004,0.996392,0.998229,0.0,0.000004,0.998229
...,...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,-1.0,-0.002092,0.928778,1.100591,0.0,-0.002092,1.018884
2019-12-30 04:00:00+00:00,1.119940,0.000018,-1.0,-0.000018,0.928794,1.100572,0.0,-0.000018,1.018866
2019-12-30 10:00:00+00:00,1.120095,0.000138,-1.0,-0.000138,0.928923,1.100420,0.0,-0.000138,1.018725


We plot returns with and without trading costs

In [40]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy_net, name='Returns (Strategy + trading costs)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [41]:
data[['returns', 'strategy_net']].mean() * (4*252)

returns        -0.035859
strategy_net    0.009180
dtype: float64

In [42]:
data[['returns', 'strategy_net']].std() * np.sqrt(4*252)

returns         0.059878
strategy_net    0.058896
dtype: float64

Now we make use of a comprehensive backtester which implements Contrarian startegy and optimize our parameters to maximize returns

In [1]:
from MomentumContrarianBacktester import MomentumContrarianBacktester as MCB

In [2]:
testMCB = MCB(symbol='EUR/USD', start='2018', end='2020', tc=0.00007, flag='c')

In [3]:
testMCB

MomentumContrarianBacktester(symbol = EUR/USD, start = 2018, end = 2020, flag = c)

In [4]:
testMCB.data

,Close,returns
time,,
2018-01-02 04:00:00+00:00,1.207055,0.004858
2018-01-02 10:00:00+00:00,1.204440,-0.002169
2018-01-02 16:00:00+00:00,1.205800,0.001129
2018-01-02 22:00:00+00:00,1.204690,-0.000921
2018-01-03 04:00:00+00:00,1.203825,-0.000718
...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092
2019-12-30 04:00:00+00:00,1.119940,0.000018
2019-12-30 10:00:00+00:00,1.120095,0.000138


In [5]:
testMCB.test_strategy()

(1.114221, 0.186048)

In [6]:
testMCB.results

,Close,returns,position,strategy,trades,strategy_net,creturns,cstrategy,cstrategy_net
time,,,,,,,,,
2018-01-02 10:00:00+00:00,1.204440,-0.002169,1.0,0.002169,0.0,0.002169,0.997834,1.002171,1.002171
2018-01-02 16:00:00+00:00,1.205800,0.001129,-1.0,0.001129,2.0,0.000989,0.998960,1.003303,1.003162
2018-01-02 22:00:00+00:00,1.204690,-0.000921,1.0,0.000921,2.0,0.000781,0.998041,1.004227,1.003946
2018-01-03 04:00:00+00:00,1.203825,-0.000718,1.0,-0.000718,0.0,-0.000718,0.997324,1.003506,1.003225
2018-01-03 10:00:00+00:00,1.202355,-0.001222,1.0,-0.001222,0.0,-0.001222,0.996106,1.002281,1.002000
...,...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,-1.0,-0.002092,0.0,-0.002092,0.927812,1.113788,0.963413
2019-12-30 04:00:00+00:00,1.119940,0.000018,-1.0,-0.000018,0.0,-0.000018,0.927828,1.113768,0.963396
2019-12-30 10:00:00+00:00,1.120095,0.000138,-1.0,-0.000138,0.0,-0.000138,0.927957,1.113614,0.963262


We find the number of trades made

In [7]:
testMCB.results.trades.value_counts()

2.0    1033
0.0    1025
1.0      10
Name: trades, dtype: int64

In [8]:
testMCB.plot_results()

We give a set of boundaries to optimize our parameters within

window_range describes the range of days over which need to be taken into consideration

In [9]:
window_range = (1, 200)

In [10]:
testMCB.optimize_parameter(window_range=window_range)

(44, 1.288223)

In [11]:
testMCB.test_strategy(window=44)

(1.288223, 0.37493)

Set of parameters which give maximum returns and a plot of returns for the same

In [12]:
testMCB.plot_results()